In [51]:
import numpy as np
import pandas as pd

In [52]:
x_train=pd.read_csv("emojify_train_x.txt",header=None)
y_train=pd.read_csv("Emojify_Y_train.txt",header=None)
x_test=pd.read_csv("emojiy_test_x.txt",header=None)
y_test=pd.read_csv("emojiy_y_test.txt",header=None)


In [53]:
y_train=[int(i) for i in y_train[0][0].split()]
y_test=[int(i) for i in y_test[0][0].split()]

In [54]:
x_train[0]=x_train[0].str.replace("'","")
x_test[0]=x_test[0].str.replace("'","")

In [55]:
import emoji 

In [56]:
emoji_dictionary={"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:"}

In [57]:
x_train

,0
0,never talk to me again
1,I am proud of your achievements
2,It is the worst day in my life
3,Miss you so much
4,food is life
5,I love you mum
6,Stop saying bullshit
7,congratulations on your acceptance
8,The assignment is too long
9,I want to go play


In [58]:
for i in range(10):
    print(x_train[0][i],emoji.emojize(emoji_dictionary[str(y_train[i])]))

never talk to me again  😓
I am proud of your achievements 😁
It is the worst day in my life  😓
Miss you so much  ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance  😁
The assignment is too long  😓
I want to go play  ⚾


In [59]:
from keras.utils import to_categorical

In [60]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [61]:
with open("glove.6B.50d.txt",encoding="utf-8") as f:
    f=f.read()

In [62]:
f.split("\n")[1].split()[0]

','

In [63]:
embedding={}
f=f.split("\n")

In [64]:
for i in range(len(f)-1):
    embedding[f[i].split()[0]]=f[i].split()[1:]

In [65]:
len(embedding["the"])

50

In [66]:
x_train[0][0][0]

'n'

In [67]:
def getembeddingmatrix(x):
    matrix=np.zeros((len(x),10,50,))
    for i in range(matrix.shape[0]):
        for j in range(len(x[i])):
            try:
                matrix[i][j]=embedding[x[i].split()[j].lower()]
            except:
                pass
    return matrix

In [68]:
embedding_matrix_train=getembeddingmatrix(x_train[0])
embedding_matrix_test=getembeddingmatrix(x_test[0])



In [69]:
from keras.layers import *
from keras.models import Sequential



In [70]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [71]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])

In [72]:
model.fit(embedding_matrix_train,y_train,epochs=40,batch_size=32,validation_split=0.1,shuffle=True)

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 3s 27ms/step - loss: 1.5852 - acc: 0.2458 - val_loss: 1.6393 - val_acc: 0.0714
Epoch 2/40
118/118 [==============================] - 0s 710us/step - loss: 1.5348 - acc: 0.3051 - val_loss: 1.6117 - val_acc: 0.1429
Epoch 3/40
118/118 [==============================] - 0s 654us/step - loss: 1.4916 - acc: 0.3983 - val_loss: 1.6261 - val_acc: 0.2143
Epoch 4/40
118/118 [==============================] - 0s 639us/step - loss: 1.4672 - acc: 0.3305 - val_loss: 1.6067 - val_acc: 0.3571
Epoch 5/40
118/118 [==============================] - 0s 695us/step - loss: 1.4131 - acc: 0.4661 - val_loss: 1.5764 - val_acc: 0.2857
Epoch 6/40
118/118 [==============================] - 0s 558us/step - loss: 1.3601 - acc: 0.4322 - val_loss: 1.5190 - val_acc: 0.3571
Epoch 7/40
118/118 [==============================] - 0s 578us/step - loss: 1.2660 - acc: 0.5169 - val_loss: 1.4576 - val_acc: 0.2857
Epoch 8/40
118/118

In [73]:
model.evaluate(embedding_matrix_test,y_test)

56/56 [==============================] - 0s 304us/step


[0.9367437958717346, 0.75]

In [74]:
model.predict_classes(embedding_matrix_test)

array([4, 3, 2, 0, 2, 2, 3, 2, 4, 2, 1, 2, 0, 3, 1, 3, 3, 2, 3, 2, 2, 0,
       4, 2, 3, 3, 1, 2, 1, 2, 0, 1, 2, 2, 3, 1, 2, 4, 4, 2, 1, 0, 0, 1,
       2, 2, 3, 2, 3, 1, 3, 0, 3, 2, 2, 0], dtype=int64)

In [76]:
for i in range(10):
    print(x_test[0][i],emoji.emojize(emoji_dictiona   ry[str(np.argmax(y_test[i]))]))
    

I want to eat  🍴
he did not answer 😓
he got a very nice raise 😁
she got me a nice present 😁
ha ha ha it was so funny 😁
he is a good friend  😁
I am upset 😓
We had such a lovely dinner tonight  😁
where is the food 🍴
Stop making this joke ha ha ha 😁
